In [1]:
# Cell 1: Setup and Parse Stock List
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from script import parse_stocks_file

ROOT = Path.cwd()
stocks_file = ROOT / 'stocks.txt'

# Parse stocks.txt to get individual tickers and baskets
individual_tickers, baskets = parse_stocks_file(stocks_file)

print(f"Individual tickers: {individual_tickers}")
print(f"\nBaskets: {baskets}")

Individual tickers: ['TSLA', 'NVDA', 'MSFT', 'META', 'PLTR', 'MSTR', 'ASML', 'AMD', 'AVGO', 'ALAB', 'MRVL', 'BTC-USD', 'SOL-USD']

Baskets: {'Main AI Basket': ['TSLA', 'NVDA', 'MSFT', 'META', 'PLTR', 'ASML'], 'Lower Risk AI Basket': ['AMD', 'AVGO', 'ALAB', 'MRVL']}


In [2]:
# Cell 2: Run Batch Analysis
from script import analyze_ticker, analyze_basket

def confluence_score(row):
    price = row['current_price']
    if price is None or pd.isna(price):
        return "UNKNOWN"
    
    inside_va = row['daily_val'] <= price <= row['daily_vah']
    at_above_poc = price >= row['daily_poc']
    above_d50 = price > row['d50']
    above_w10 = price > row['w10']
    
    if inside_va and at_above_poc and above_d50 and above_w10:
        return "Bullish / Favor Add"
    elif inside_va:
        return "Neutral / Acceptable"
    else:
        return "Caution / Skip"  # or Bearish/Pause

def recommendation(row):
    signal = row['signal']
    confluence = row.get('confluence', 'UNKNOWN')
    if "FULL HOLD + ADD" not in signal:
        return "No Buy"
    if confluence == "Bullish / Favor Add":
        return "No Starter – Wait for Primary Dip"
    elif confluence == "Neutral / Acceptable":
        return "Immediate Starter Possible on Minor Dip"
    else:
        return "No Buy – Wait for Better Confluence"

def run_batch(individual_tickers, baskets, concurrency=6, daily_bars=60, weekly_bars=52):
    """Run analysis on all individual tickers and baskets"""
    results = []
    
    # Analyze individual tickers in parallel
    with ThreadPoolExecutor(max_workers=concurrency) as ex:
        futures = {ex.submit(analyze_ticker, t, daily_bars, weekly_bars): t for t in individual_tickers}
        for fut in as_completed(futures):
            results.append(fut.result())
    
    # Analyze baskets (market cap weighted aggregations)
    for basket_name, constituents in baskets.items():
        basket_result = analyze_basket(basket_name, constituents, daily_bars, weekly_bars)
        results.append(basket_result)
    
    df = pd.DataFrame(results)
    
    # Add confluence score and recommendation columns
    df['confluence'] = df.apply(confluence_score, axis=1)
    df['recommendation'] = df.apply(recommendation, axis=1)
    
    return df

# Run the analysis
df = run_batch(individual_tickers, baskets)
df = df.round(2)

# Print Summary Stats
print(f"\nAnalysis complete: {len(df)} rows ({len(individual_tickers)} tickers + {len(baskets)} baskets)")
print("FULL HOLD + ADD tickers:", ', '.join(df[df['signal'] == 'FULL HOLD + ADD']['ticker']))

df


Analysis complete: 15 rows (13 tickers + 2 baskets)
FULL HOLD + ADD tickers: PLTR, TSLA, NVDA, ASML, MRVL, [Main AI Basket]


,ticker,signal,current_price,price_note,date,d20,d50,d100,d200,w10,...,weekly_val,s1,s2,s3,r1,r2,r3,notes,confluence,recommendation
0,PLTR,FULL HOLD + ADD,167.86,last close,2026-01-04,184.55,180.94,176.47,151.04,179.10,...,69.16,147.56,142.34,128.51,207.52,190.00,188.20,,Caution / Skip,No Buy – Wait for Better Confluence
1,TSLA,FULL HOLD + ADD,438.07,last close,2026-01-04,464.27,445.01,418.45,360.20,442.77,...,219.94,411.45,382.78,373.04,488.54,474.07,470.75,,Neutral / Acceptable,Immediate Starter Possible on Minor Dip
2,MSTR,CASH,157.16,last close,2026-01-04,167.75,202.40,267.75,320.20,179.39,...,235.84,155.61,120.23,113.69,543.00,457.22,430.35,,Neutral / Acceptable,No Buy
3,MSFT,HOLD MOST → REDUCE,472.94,last close,2026-01-04,483.17,495.21,503.04,476.73,485.44,...,410.33,464.89,407.71,404.37,553.50,552.69,530.04,,Neutral / Acceptable,No Buy
4,META,HOLD MOST → REDUCE,650.41,last close,2026-01-04,658.59,651.82,698.45,672.61,640.47,...,586.28,580.78,578.18,546.88,795.06,789.62,758.54,,Neutral / Acceptable,No Buy
5,NVDA,FULL HOLD + ADD,188.85,last close,2026-01-04,182.96,186.50,183.02,160.48,183.82,...,86.60,176.75,169.54,164.05,212.18,NaN,NaN,,Bullish / Favor Add,No Starter – Wait for Primary Dip
6,AVGO,HOLD,347.62,last close,2026-01-04,360.65,361.22,344.91,288.92,359.56,...,220.24,328.43,323.43,280.85,413.82,385.74,372.88,,Neutral / Acceptable,No Buy
7,ASML,FULL HOLD + ADD,1163.78,last close,2026-01-04,1083.21,1056.14,965.47,843.33,1063.95,...,574.25,988.40,946.11,933.75,NaN,NaN,NaN,,Caution / Skip,No Buy – Wait for Better Confluence
8,AMD,HOLD,223.47,last close,2026-01-04,214.41,227.63,203.23,163.23,219.71,...,129.85,194.28,161.81,153.34,267.08,227.30,NaN,,Neutral / Acceptable,No Buy
9,ALAB,HOLD MOST → REDUCE,179.56,last close,2026-01-04,162.40,160.76,179.74,134.57,161.72,...,107.54,148.51,131.42,84.78,262.90,201.86,199.47,,Caution / Skip,No Buy


In [3]:
# Cell 3: Export Results
import shutil
from datetime import datetime

# Add timestamp to avoid overwriting
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_file = ROOT / f'batch_results_{timestamp}.csv'

# Save to workspace
df.to_csv(output_file, index=False)
print(f"✅ Results exported to: {output_file}")

# Copy to Downloads folder
downloads_path = Path.home() / 'Downloads' / f'batch_results_{timestamp}.csv'
try:
    shutil.copy(output_file, downloads_path)
    print(f"✅ File also copied to: {downloads_path}")
except PermissionError:
    print(f"⚠️  Could not copy to Downloads - file may be open in another program")
    print(f"   Close the file and try again, or use: {output_file}")

✅ Results exported to: c:\workspace\my_script_project\batch_results_20260104_1156.csv
✅ File also copied to: C:\Users\karms\Downloads\batch_results_20260104_1156.csv
